In [ ]:
import re

from bs4 import BeautifulSoup
import ndjson
import requests
import tqdm

with open("input.ndjson") as f:
    data = ndjson.load(f)

peak_rows = [row for row in data if row["_type"] == "peak"]

for row in tqdm.tqdm(peak_rows):
    link = row.get("link")
    if link is None:
        continue
    response = requests.get(link)
    soup = BeautifulSoup(response.text, "html.parser")
    tags = iter(soup.find_all("td"))
    for tag in tags:
        if tag.text == "Latitude/Longitude (WGS84)":
            break
    tag = next(tags)
    match = re.match(
        "^(.*), (.*) \(Dec Deg\)$",
        list(tag.stripped_strings)[1]
    )
    lat, lng = map(float, match.groups())
    row["location"] = {
        "lat": lat,
        "lng": lng
    }

with open("output.ndjson", "w") as f:
    ndjson.dump(data, f)